# High-Performance Pandas: eval() and query()

As we´ve already seen in previous sections, the power of the PyData stack is built upong the ability of NumPy and Pandas to push basic operations into C via and intuitive syntax: examples are vectorized/broadcasted operations in NumPy, and grouping-type operations in Pandas. While these abstractions are efficient and effective for many common use cases, they often rely on the creation of temporary intermediate objects, which can cause undue overhead in computational time and memory use.

Pandas includes some experimental tools that allow you to directly access C-speed operations without costly allocation of intermediate arrays. These are the `eval()` and `query()` functions, which rely on the [Numexpr](https://github.com/pydata/numexpr) package. In this notebook we will walk through their use and give some rules-of-thumb about when you might think about using them.

## Motivating `query()` and `eval()`: Compound Expressions
We've seen previously that NumPy and Pandas support fast vectorized operations; for example, when adding the elements of two arrays:

In [1]:
import numpy as np 
rng = np.random.RandomState(4)
x = rng.rand(1_000_000)
y = rng.rand(1_000_000)
%timeit x + y

9.42 ms ± 680 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


As discussed, this is much faster than doing the addition via a Python loop or comprehension:

In [2]:
%timeit np.fromiter((xi + yi for xi, yi in zip(x, y)), dtype=x.dtype, count=len(x))

608 ms ± 148 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


But this abstraction can become less efficient when computing compound expressions. For example, consider the following expression:

In [3]:
mask = (x > 0.5) & (y < 0.5)

Because NumPy evaluates each subexpression, this is roughly equivalent to the following:

In [4]:
tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2

In other words, *every intermediate step is explicitly allocated in memory*. If the `x` and `y` arrays are very large, this can lead to significant memory and computational overhead. The Numexpr library gives you the ability to compute this type of compound expression element by element, without the need to allocate full intermediate a arrays. The Numexpr documentation has more details, but for the time being it is sufficient to say that the library accepts a *string* giving the NumPy-style expression you'd like to compute:

In [6]:
import numexpr
mask_numexpr = numexpr.evaluate('(x > 0.5) & (y < 0.5)')
np.allclose(mask, mask_numexpr)

True

The benefit here is that Numexpr evaluates the expression in a way that does not use full-size temporary arrays, and thus can be much more efficient than NumPy, especially for large arrays. The Pandas `eval()` and `query()` tools that we will discuss here are conceptually similar, and depend on the Numexpr package.

## `pandas.eval()` for Efficient Operations

The `eval()` function in Pandas uses string expressions to efficiently compute operations using `DataFrame`s. For example, consider the following `DataFrame`s:

In [7]:
import pandas as pd 
nrows, ncols = 100_000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols)) for i in range(4))

To compute hte sum of all four `DataFrame`s using the typical pandas approach, we can just write the sum:

In [8]:
%timeit df1 + df2 + df3 + df4

101 ms ± 15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%timeit pd.eval('df1 + df2 + df3 + df4')

34.8 ms ± 929 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


The `eval()` version of this expression is about 50% faster (and uses much less memory), while giving the same result:

In [10]:
np.allclose(df1 + df2 + df3 + df4, pd.eval('df1 + df2 + df3 + df4'))

True

### Operations supported by `pd.eval()`
As of Pandas v0.16, `pd.eval()` supports a wide range of oeprations. To demostrate these, we'll use the following integer `DataFrame`s:

In [11]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 100, (100, 3))) for i in range(5))

#### Arithmetic operators
`pd.eval()` supports all arithmetic operators. For example:


In [14]:
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

True

#### Comparison operators
`pd.eval()` supports all comparison operators, including chained expressions:

In [16]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

True

#### Bitwise operators
`pd.eval()` supports the & and | bitwise operators: 

In [17]:
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

In addition, it supports the use of the literal `and` and `or` in Boolean expressions:

In [19]:
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

True

#### Object attributes and indices
`pd.eval()` supports access to object attributes via the `obj.attr` syntax, and indexes via the `obj[index]` syntax:

In [20]:
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

#### Other operations
Other operations such as function calls, conditional statements, loops, and other more involved construct are currently *not* implemented in `pd.eval()`. If you'd like to execute these more complicated types of expressions, you can use the Numexpr library itself.

## `DataFrame.eval()` for Column-Wise Operations
Just as Pandas has a top-level `pd.eval()` function, `DataFrame`s have and `eval()` method that owrks in similar ways. The benefit of the `eval()` method is taht columns can be referred to be *name*. We'll use this labeld array as an example:

In [24]:
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df.head()

,A,B,C
0,0.763183,0.777655,0.938027
1,0.871498,0.765464,0.971125
2,0.332477,0.170807,0.105295
3,0.934971,0.417575,0.371057
4,0.671115,0.153091,0.246334


Using `pd.eval()` as above, we can compute expressions with the three columns like this:

In [25]:
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")
np.allclose(result1, result2)

True

The `DataFrame.eval()` method allows much more succinct evaluation of expressions with the columns:

In [27]:
result3 = df.eval('(A + B) / (C - 1)')
np.allclose(result1, result3)

True

Notice here that we treat *column names as variables* within the evaluated expression, and the result is what we would wish.

### Assignment in DataFrame.eval()
In additino to the options just discussed, `DataFrame.eval()` also allows assignment to any column. Let's use the `DataFrame` from before, which has columns `'A'`, `'B'`, and `'C'`:

In [29]:
df.head()

,A,B,C
0,0.763183,0.777655,0.938027
1,0.871498,0.765464,0.971125
2,0.332477,0.170807,0.105295
3,0.934971,0.417575,0.371057
4,0.671115,0.153091,0.246334


We can use `df.eval()` to create a new column `D` and assign to it a value computed fomr hte other columns:

In [30]:
df.eval('D = (A + B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.763183,0.777655,0.938027,1.642638
1,0.871498,0.765464,0.971125,1.685635
2,0.332477,0.170807,0.105295,4.779742
3,0.934971,0.417575,0.371057,3.645121
4,0.671115,0.153091,0.246334,3.345893


In the same way, any existing column can be modified:

In [31]:
df.eval('D = (A - B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.763183,0.777655,0.938027,-0.015428
1,0.871498,0.765464,0.971125,0.109187
2,0.332477,0.170807,0.105295,1.535398
3,0.934971,0.417575,0.371057,1.394388
4,0.671115,0.153091,0.246334,2.102939


### Local variables in DataFrame.eval()
The `DataFrame.eval()` method supports an additional sntax that lets it work with local Python variables, Consider the following:

In [32]:
column_mean = df.mean(1)
result1 = df['A'] + column_mean 
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)

True

The `@` character here marks a *variable name* rather than a *column name*, and lets you efficiently evaluate expressions involving the two "namespaces": the namespace of columns, and the namespace of Python objects. Notice that this `@` character is only supported by the `DataFrame.eval()` *method*, not by the `pandas.eval()` *function*, becasue the `pandas.eval()` function only has access to the one (Python) namespace.

## DtaFrame.query() Method
The `DataFrame` has another method based on evaluated strings, valled the `query()` method. Consider the following:

In [35]:
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)

True

As with the example used in our discussion of `DataFrame.eval()`, this is an expression involing columns of the `DataFrame`. It cannot be expressed using the `DataFrame.eval()` syntax, however! Instead, for this type of filtering operation, you can use the `query()` method:

In [36]:
result2 = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result2)

True

In addition to being a more efficient computatino, compared to the masking expression this is much easier to read and understand. Note that the `query()` mehtod also accepts the `@` flag to mark local variables:

In [37]:
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean and B < @Cmean')
np.allclose(result1, result2)

True